### Limpieza general 
normalizar fondo y eliminar archivos no finalizados en .png

In [4]:
import os
import io
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd

version="1.21.1"
tam=32


item_folder = f"../datos/item_{version}_{str(tam)}x{str(tam)}"

BACKGROUND_COLOR = (255, 255, 255)   

###  Definición de Funciones

In [5]:
def safe_open_image(path):
    try:
        im = Image.open(path)
        im.load()
        return im
    except Exception as e:
        print(f"ERROR abriendo {path}: {e}")
        return None

def ensure_rgb_with_bg(im, background_color=BACKGROUND_COLOR):
    """
    Convierte P/RGBA a RGB con fondo.
    """
    if im.mode == "P":
        im = im.convert("RGBA")

    if im.mode == "RGBA":
        bg = Image.new("RGB", im.size, background_color)
        bg.paste(im, mask=im.split()[-1])  # usa el alpha como máscara
        im = bg
    else:
        im = im.convert("RGB")
    return im

def dataset_from_folder(folder_path: str):

    image_files = []
    # Listar archivos, ignorando posibles errores si no existe la carpeta
    if os.path.exists(folder_path):
        for f in os.listdir(folder_path):
            image_files.append(f)
    else:
        print(f"Advertencia: La carpeta {folder_path} no existe.")
        return pd.DataFrame()

    images = []
    labels = []

    for filename in sorted(image_files):
        image_path = os.path.join(folder_path, filename)

        # abrir imagen 
        im = safe_open_image(image_path)
        if im is None:
            continue
            
        im = ensure_rgb_with_bg(im)

        buf = io.BytesIO()
        im.save(buf, format="PNG")

        images.append({"bytes": buf.getvalue()})

        # --- generar label desde el nombre ---
        label = os.path.splitext(filename)[0].replace("_", " ")
        labels.append(label)

    # --- dataframe final ---
    df = pd.DataFrame({
        "image": images,
        "label": labels
    })

    return df

###  Items

In [6]:
# eliminamos el archivo compass 14 de items pq da problemas
compass_path = os.path.join(item_folder, "compass_14.png")

os.remove(compass_path)
print(f"Eliminado: {compass_path}")

# Cargar dataset de items
df_items = dataset_from_folder(folder_path=item_folder)
df_items.head()

Eliminado: ../datos/item_1.21.1_32x32\compass_14.png


,image,label
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia boat
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia chest boat
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia door
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia hanging sign
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia sign


In [8]:
output_path = f"../datos/faithful_{version}_{str(tam)}x{str(tam)}.parquet"
df_items.to_parquet(output_path)
print(f"Dataset guardado en {output_path} con {len(df_items)} imágenes.")

Dataset guardado en ../datos/faithful_1.21.1_32x32.parquet con 603 imágenes.
